# Lab 19: Persistent Memory with Redis - Setup and Initial Conversations

This lab demonstrates how to implement persistent memory in LangChain using Redis as a data store. You'll learn:
- How to set up Redis-based chat message history
- Using `RedisChatMessageHistory` for persistent conversation storage
- Managing multiple conversation threads with session IDs
- Building conversational AI with long-term memory capabilities
- Understanding the difference between in-memory and persistent storage

In [ ]:
# Import LangChain components for Redis-based persistent memory implementation
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
# Redis-specific imports for persistent chat history
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import os
# Configure OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key"

# Initialize ChatOpenAI model for conversational interactions
model = ChatOpenAI()

# Create chat prompt template with memory placeholder
# MessagesPlaceholder will be populated with conversation history from Redis
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),  # History from Redis
        ("human", "{input}"),  # Current user input
    ]
)

# Create base chain without memory integration yet
base_chain = prompt | model

In [ ]:
# Configure Redis connection parameters
# Default Redis URL for local Redis instance on port 6379, database 0
REDIS_URL = "redis://localhost:6379/0"

# Initialize empty store (not used in this Redis implementation)
# This variable exists for compatibility but Redis handles storage directly
store = {}

In [ ]:
# Define function to create Redis chat message history for each session
# Each session_id creates a separate conversation thread in Redis
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    """Create or retrieve Redis-based message history for a specific session"""
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

# Create a runnable chain with persistent Redis-based message history
# This wraps our base chain with memory management capabilities
redis_chain = RunnableWithMessageHistory(
    base_chain,                          # The base conversational chain
    get_message_history,                 # Function to get/create message history
    input_messages_key="input",          # Key for current user input
    history_messages_key="history",      # Key for injecting conversation history
)

In [ ]:
# Start the first conversation in "math-thread1" session
# This creates a new conversation thread and stores it in Redis
# The session_id uniquely identifies this conversation for future retrieval
redis_chain.invoke(
    {"ability": "math", 
     "input": "What does cosine mean?"}, 
    config={"configurable": {"session_id": "math-thread1"}},  # Creates persistent thread
)

In [ ]:
# Continue the conversation in the same "math-thread1" session
# Redis retrieves previous messages, so "Tell me more!" has context about cosine
# The AI can reference the previous question about cosine
redis_chain.invoke(
    {"ability": "math", "input": "Tell me more!"},
    config={"configurable": {"session_id": "math-thread1"}},  # Same thread continues
)

In [ ]:
# Start a completely separate conversation in "phy-thread1" session
# This demonstrates multiple independent conversation threads in Redis
# Different session_id = different conversation context
redis_chain.invoke(
    {"ability": "physics", 
     "input": "What is the theory of relativity?"}, 
    config={"configurable": {"session_id": "phy-thread1"}},  # New independent thread
)

In [ ]:
# Continue the physics conversation in "phy-thread1" session
# "Tell me more!" now refers to relativity theory, not cosine
# Each session maintains separate conversation context in Redis
redis_chain.invoke(
    {"ability": "physics", "input": "Tell me more!"},
    config={"configurable": {"session_id": "phy-thread1"}},  # Physics thread continues
)